Классы объектов:
- Клиенты - **User** (Название - `name`, Вид собственности - `property`, Адрес - `address`, Телефон - `phone`, Контактное лицо - `contact_person`).
- Виды кредитов - **Credit** (Название - `name`, Условия получения - `description`, Ставка - `rate`, Срок - `period`).
- Кредиты - **Contract** (Вид кредитов - `credit_id`, Клиент - `user_id`, Сумма - `amount`, Дата выдачи - `start_date`).


In [1]:
from dataclasses import dataclass

import random
from faker import Faker
fake_data = Faker("ru_RU")

import sqlite3
from sqlite3 import Error

import pandas as pd

# Запросы и подключение к БД

In [2]:
database = "credits.db"

# CREATE

sql_create_users_table = """
                        CREATE TABLE IF NOT EXISTS users (
                            id integer PRIMARY KEY AUTOINCREMENT,
                            name text NOT NULL,
                            property text,
                            address text,
                            phone text,
                            contact_person text                         
                        ); """

sql_create_credits_table = """
                        CREATE TABLE IF NOT EXISTS credits (
                            id integer PRIMARY KEY AUTOINCREMENT,
                            name text NOT NULL,
                            description text,
                            rate real NOT NULL,
                            period integer
                        ); """

sql_create_contracts_table = """
                        CREATE TABLE IF NOT EXISTS contracts  (
                            id integer PRIMARY KEY AUTOINCREMENT,
                            credit_id integer,
                            user_id integer,
                            amount integer,
                            start_date text,
                            FOREIGN KEY (credit_id) REFERENCES credits (id),
                            FOREIGN KEY (user_id) REFERENCES users (id)
                        ); """

# INSERT

sql_insert_users_table = """ INSERT INTO users (name, property, address, phone, contact_person) VALUES (?,?,?,?,?); """

sql_insert_credits_table = """ INSERT INTO credits (name, description, rate, period) VALUES(?,?,?,?); """

sql_insert_contracts_table = """ INSERT INTO contracts (credit_id, user_id, amount, start_date) VALUES(?,?,?,?); """

# SELECT

sql_select_users_table = """ SELECT * FROM users; """

sql_select_credits_table = """ SELECT * FROM credits; """

sql_select_contracts_table = """ SELECT * FROM contracts; """

sql_select_user_by_id = """ SELECT * FROM users WHERE id = ?; """

sql_select_credit_by_id = """ SELECT * FROM credits WHERE id = ?; """

sql_select_contract_by_id = """ SELECT * FROM contracts WHERE id = ?; """

# UPDATE

sql_update_credit = """ UPDATE credits
                        SET name = ?,
                            description = ?,
                            rate = ?,
                            period = ?
                        WHERE id = ?; """

sql_update_user = """   UPDATE users
                        SET name = ?,
                            property = ?,
                            address = ?,
                            phone = ?,
                            contact_person = ?
                        WHERE id = ?; """

# DELETE

sql_delete_credit = """ DELETE FROM credits WHERE id = ?; """

sql_delete_user = """ DELETE FROM users WHERE id = ?; """

In [3]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return conn


def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [4]:
conn = create_connection(database)


create_table(conn, sql_create_users_table)
create_table(conn, sql_create_credits_table)
create_table(conn, sql_create_contracts_table)

# Функции obj↔db

## obj → db

In [5]:
# INSERT

def insert_users_table(conn, obj):
    try:
        cur = conn.cursor()
        user = (obj.name, obj.property, obj.address, obj.phone, obj.contact_person)
        cur.execute(sql_insert_users_table, user)
        conn.commit()
        return cur.lastrowid
    except Error as e:
        print(e)

def insert_credits_table(conn, obj):
    try:
        cur = conn.cursor()
        credit = (obj.name, obj.description, obj.rate, obj.period)
        cur.execute(sql_insert_credits_table, credit)
        conn.commit()
        return cur.lastrowid
    except Error as e:
        print(e)

def insert_contracts_table(conn, obj):
    try:
        cur = conn.cursor()
        contract = (obj.credit_id, obj.user_id, obj.amount, obj.start_date)
        cur.execute(sql_insert_contracts_table, contract)
        conn.commit()
        return cur.lastrowid
    except Error as e:
        print(e)

# UPDATE

def update_credit(conn, obj):
    try:
        cur = conn.cursor()
        credit = (obj.name, obj.description, obj.rate, obj.period, obj._id)
        cur.execute(sql_update_credit, credit)
        conn.commit()
    except Error as e:
        print(e)

def update_user(conn, obj):
    try:
        cur = conn.cursor()
        user = (obj.name, obj.property, obj.address, obj.phone, obj.contact_person, obj._id)
        cur.execute(sql_update_user, user)
        conn.commit()
    except Error as e:
        print(e)

# DELETE

def delete_credit(conn, obj):
    try:
        cur = conn.cursor()
        cur.execute(sql_delete_credit, (obj._id, ))
        conn.commit()
    except Error as e:
        print(e)

def delete_user(conn, obj):
    try:
        cur = conn.cursor()
        cur.execute(sql_delete_user, (obj._id, ))
        conn.commit()
    except Error as e:
        print(e)



## obj ← db

In [ ]:
# SELECT

def select_user_by_id(conn, id):
    try:
        cur = conn.cursor()
        cur.execute(sql_select_user_by_id, (id, ))
        conn.commit()
        return cur.fetchone()
    except Error as e:
        print(e)

def select_credit_by_id(conn, id):
    try:
        cur = conn.cursor()
        cur.execute(sql_select_credit_by_id, (id, ))
        conn.commit()
        return cur.fetchone()
    except Error as e:
        print(e)

def select_contract_by_id(conn, id):
    try:
        cur = conn.cursor()
        cur.execute(sql_select_contract_by_id, (id, ))
        conn.commit()
        return cur.fetchone()
    except Error as e:
        print(e)

# Классы

In [ ]:
# Название, Вид собственности, Адрес, Телефон, Контактное лицо
@dataclass
class User:
    _id: int
    name: str
    property: str
    address: str
    phone: str
    contact_person: str

    def __init__(
        self,
        name=None,
        property=None,
        address=None,
        phone=None,
        contact_person=None,
        get_by_id=None,
    ):
        if get_by_id == None:
            if name == None:
                self.name = fake_data.name()
            else:
                self.name = name

            if property == None:
                self.property = fake_data.bs()
            else:
                self.property = property

            if address == None:
                self.address = fake_data.address()
            else:
                self.address = address

            if phone == None:
                self.phone = fake_data.phone_number()
            else:
                self.phone = phone

            if contact_person == None:
                self.contact_person = fake_data.name()
            else:
                self.contact_person = contact_person

            self._count = 0

            self._id = insert_users_table(conn, self)

        else:  # добавить проверку на корректность id
            user = select_user_by_id(conn, get_by_id)
            self._id = user[0]
            self.name = user[1]
            self.property = user[2]
            self.address = user[3]
            self.phone = user[4]
            self.contact_person = user[5]

    def delete(self):
        delete_user(conn, self)

    def update(self, name=None, age=None):
        if name != None:
            self.name = name
            update_user(conn, self)

        if age != None:
            self.age = age
            update_user(conn, self)


# Название, Условия получения, Ставка, Срок
@dataclass
class Credit:
    _id: int
    name: str
    description: str
    rate: float
    period: int

    def __init__(
        self, 
        name=None, 
        description=None, 
        rate=None, 
        period=None, 
        get_by_id=None
    ):
        if get_by_id == None:
            if name == None:
                self.name = fake_data.paragraph(nb_sentences=1)
            else:
                self.name = name

            if description == None:
                self.description = fake_data.paragraph(nb_sentences=5)
            else:
                self.description = description

            if rate == None:
                self.rate = round(random.uniform(0, 1), 2)
            else:
                self.rate = rate

            if period == None:
                self.period = random.randint(1, 365)
            else:
                self.period = period

            self._id = insert_credits_table(conn, self)

        else:
            credit = select_credit_by_id(conn, get_by_id)
            self._id = credit[0]
            self.name = credit[1]
            self.description = credit[2]
            self.rate = credit[3]
            self.period = credit[4]


# Вид кредитов, Клиент, Сумма, Дата выдачи
@dataclass
class Contract:
    _id: int
    credit_id: int
    user_id: int
    amount: int
    start_date: str

    def __init__(
        self, 
        credit_id=None, 
        user_id=None, 
        amount=None, 
        start_date=None, 
        get_by_id=None
    ):
        if get_by_id == None:
            if credit_id == None:
                credit_id_list = pd.read_sql(sql_select_credits_table, conn)["id"].tolist()
                self.credit_id = random.choice(credit_id_list)
            else:
                self.credit_id = credit_id

            if user_id == None:
                user_id_list = pd.read_sql(sql_select_users_table, conn)["id"].tolist()
                self.user_id = random.choice(user_id_list)
            else:
                self.user_id = user_id

            if amount == None:
                self.amount = random.randint(100000, 5000000)
            else:
                self.amount = amount

            if start_date == None:
                self.start_date = fake_data.date()
            else:
                self.start_date = start_date

            self._id = insert_contracts_table(conn, self)

        else:
            contract = select_contract_by_id(conn, get_by_id)
            self._id = contract[0]
            self.credit_id = contract[1]
            self.user_id = contract[2]
            self.amount = contract[3]
            self.start_date = contract[4]

    def info(self):
        return (
            self,
            # select_user_by_id(conn, self.user_id), 
            User(get_by_id = self.user_id),
            # select_credit_by_id(conn, self.credit_id), 
            Credit(get_by_id = self.credit_id),
        )


# Ввод данных

In [17]:
u = User()
u

User(_id=1, name='Капустина Василиса Аркадьевна', property='Трансформация круглосуточных интернет-магазинов', address='ст. Верхотурье, алл. Кошевого, д. 5/9, 071826', phone='+7 (183) 713-2624', contact_person='Куликова Вероника Владиславовна')

In [18]:
k = Credit()
k

Credit(_id=1, name='Уничтожение висеть мимо призыв ребятишки поезд.', description='Темнеть написать выгнать падать плавно очутиться степь. Господь жить пол сходить витрина. Назначить потом космос болото. Невыносимый пища угодный выбирать головной. Сверкающий место страсть функция школьный.', rate=0.06, period=196)

In [30]:
c = Contract()
c

Contract(_id=3, credit_id=1, user_id=1, amount=2018233, start_date='2001-10-05')

In [ ]:
c.info()

Contract(_id=1, credit_id=1, user_id=1, amount=4718006, start_date='1997-05-20')

# Проверка 

## Клиенты

In [20]:
pd.read_sql(sql_select_users_table, conn)

,id,name,property,address,phone,contact_person
0,1,Капустина Василиса Аркадьевна,Трансформация круглосуточных интернет-магазинов,"ст. Верхотурье, алл. Кошевого, д. 5/9, 071826",+7 (183) 713-2624,Куликова Вероника Владиславовна


## Виды кредитов

In [21]:
pd.read_sql(sql_select_credits_table, conn)

,id,name,description,rate,period
0,1,Уничтожение висеть мимо призыв ребятишки поезд.,Темнеть написать выгнать падать плавно очутить...,0.06,196


## Выданные кредиты

In [32]:
pd.read_sql(sql_select_contracts_table, conn)

,id,credit_id,user_id,amount,start_date
0,1,1,1,4718006,1997-05-20
1,2,1,1,139002,2017-03-08
2,3,1,1,2018233,2001-10-05


# UI компоненты

## Web app

Streamlit = Web app!

## Desktop app

https://discuss.streamlit.io/t/streamlit-wasm-electron-desktop-app/31655

Streamlit + Wasm + Electron = Desktop app!

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f982d86a-fb45-4a02-b4ea-c7285dcef6ac' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>